In [ ]:
#Imports of all necessary Python Packages
import sklearn as skl
import numpy as np
import pandas as pd
from time import time

#Classifiers
from sklearn.svm import SVC
from sklearn.svm import NuSVC

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.decomposition import PCA

from sklearn.dummy import DummyClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegressionCV


#Evaluation

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn import cross_validation
from sklearn.cross_validation import train_test_split

In [ ]:
#Here import the data to classify

pos_tags = pd.read_csv('pos_tags_train_data.csv', index_col=0)
pos_tags 
labels = pos_tags['labels'].values
data = pos_tags.drop('labels', axis=1).values
print data.shape
print labels.shape

In [ ]:
t0 = time()
# Feature Extraction
pca = PCA(n_components = 500)
data_decomposed = pca.fit_transform(data)
print("done in %0.3fs." % (time() - t0))

In [ ]:
#Generate a Few of the most comon Scikit-Learn Classifiers
#SVM
classfSVM = SVC(C=1, kernel='rbf', gamma='auto', tol=1e-3) #53%
#SVM with control of the %(nu) of support vectors 
classfNuSVM = NuSVC(nu=0.5, kernel='rbf', gamma='auto', tol=1e-3)
#LDA
classfLDA = LinearDiscriminantAnalysis()
#QDA
classfQDA = QuadraticDiscriminantAnalysis()
#KNN
classfKNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)#34%
#DecisionTree
classfDecisionTree = DecisionTreeClassifier()
#AdaBoost
classfAdaBoost = AdaBoostClassifier(base_estimator=None, learning_rate=1, n_estimators=50)
#RandomForest
classfRandForest = RandomForestClassifier(n_estimators=30, n_jobs=-1)

#LogisticRegressionCV
classfLogRegressCV = LogisticRegressionCV(cv = 5, n_jobs=-1)

classf_lst = [classfLDA, classfQDA, classfDecisionTree, classfAdaBoost, classfRandForest]

In [ ]:
#Test classifier on all data or preferably on a Test set:
#DON'T trust these results, there is an high chance of overfitting
#Use ONLY to get a feel of the time Cross-Validation will take

classf = classfKNN
train_data, test_data, train_labels, test_labels = train_test_split(data_decomposed,
                                                                    labels, test_size=0.20, random_state=42)

t0 = time()
classf.fit(train_data,train_labels)
print "Score of Classifier: " + str(classf.score(test_data,test_labels))
print("done in %0.3fs." % (time() - t0))

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(data_decomposed,
                                                                    labels, test_size=0.20, random_state=42)
for classf in classf_lst:
    print classf
    t0 = time()
    classf.fit(train_data,train_labels)
    print "Score of Classifier: " + str(classf.score(test_data,test_labels))
    print("done in %0.3fs." % (time() - t0))

In [ ]:
# Analyze Confusion Matrix
#test_labels = labels
#test_data = data
#train_data = data
#train_labels

#classf = classfSVM
#classf.fit(train_data)
predicted_labels = classf.predict(test_data)
confusion_matrix(test_labels, predicted_labels)

In [ ]:
# Select parameters to use in Cross-Validation
classf_cv = classfKNN
data_cv = data_decomposed
N_CV = 10

# Cross Validation
t0 = time()
scores = cross_validation.cross_val_score(classf_cv,data_cv,labels, n_jobs=-1, cv = N_CV)
print "Scores: "
for i,score in enumerate(scores):
    print '\t' + str(i) + ':\t' + str(score) 
print "\nCrossval Final score: "+ str(sum(scores)/N_CV)
print("\nCross val done in %0.3fs." % (time() - t0))